In [1]:
import nemo

In [1]:
from __future__ import print_function, division,unicode_literals
import os
import pandas as pd
# from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import librosa

from io import open
import unicodedata
import string
import re
import random

import tqdm
import json

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch import optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data.dataloader import default_collate


import torch.nn.functional as Fi


import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)


import string

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1.5.1
0.5.0a0+738ccba


################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
table_trans = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
max_duration =8


# path ="/Users/dami.osoba/work/bawk/small_dataset/small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv"
# path = "../../commonvoice_validated_manifest.json"
# meta = pd.read_csv(path,sep="\t")
# meta_path = meta.set_index('path')

def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm.tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest
# train_manifest_path = '/Users/dami.osoba/work/bawk/src/data/commonvoice_train_manifest.json'
train_manifest_path = "../../commonvoice_validated_manifest.json"
train_manifest_data = read_manifest(train_manifest_path)
# keep audio < 4s
current = os.listdir('../../wav_clips/')
train_text = [data['text'] for data in train_manifest_data if data['duration']<=8]

train_path = [(data['audio_filepath'].replace('validated','wav_clips'), data['text']) for data in train_manifest_data if data['duration'] <= max_duration]

train_path_pd = pd.DataFrame(train_path, columns=['train_path','sentence'])

# remove unicode
sentences = [c.encode(encoding="ascii",errors="ignore").decode().translate(table_trans) for c in train_text]
char_dict = sorted(list(set([b for a in sentences for b in a]))) +['EOS','SOS','PAD']
char_index = {a:char_dict.index(a) for a in char_dict}
dictOfindex = {char_dict.index(a):a for a in char_dict}
dictOfindex = { i : char_dict[i] for i in range(0, len(char_dict) ) }
dictOfchar = { char_dict[i]:i for i in range(0, len(char_dict) ) }
# char_index['EOS'] = len(char_dict)
# char_index['SOS'] = len(char_dict)+1
# char_index['PAD'] = len(char_dict)+2

Reading manifest data: 1224863it [00:04, 280296.78it/s]


In [4]:
dev

1151580

# Create voice dataset

In [3]:
class VoiceDataset(Dataset):
    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         self.landmarks_frame = pd.read_csv(csv_file)
#         self.root_dir = root_dir
        self.transform = transform
        self.path_frame = train_path_pd

    def __len__(self):
        return len(self.path_frame)

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        waveform, _ = librosa.load(self.path_frame.loc[idx][0],sr=16000)
#         waveform, _ = torchaudio.load(self.path_frame.loc[idx][0],)
        # transcription for audio
        trans = self.path_frame.loc[idx][1]
        # encode to ascii
        trans = trans.encode(encoding="ascii",errors="ignore").decode().translate(table_trans).lower()
        chars =[b for a in trans for b in a]
#         coded = [28]+[char_dict.index(a) for a in chars]+[27]
        coded = [char_dict.index(a) for a in chars]


        sample = {'waveform': waveform, 'transcription': coded,'sentence':trans}

        if self.transform:
            sample = self.transform(sample)        

        return sample


# Create FFT transform

In [4]:
window_size = 25/1000
stride = 10/1000
sample_rate = 16000
n_fft =int(window_size *sample_rate)
win_length = None
hop_length = int(sample_rate*stride)
n_mels = 80
max_time = 8

class MelSpec(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self):        
        self.window_size = 25/1000
        self.stride = 10/1000
        self.sample_rate = 16000
        self.n_fft =int(self.window_size *self.sample_rate)
        self.win_length = None
        self.hop_length = int(self.sample_rate*self.stride)
        self.n_mels = 80
        self.max_time = 8

    def mel_spectrogram(self,a):    
        mel_spec = librosa.feature.melspectrogram(a,
                                                  sr=self.sample_rate,
                                                  n_fft = self.n_fft,
                                                  hop_length =self.hop_length,
                                                  center =True,
                                                  n_mels=self.n_mels,
                                                  htk=True,
                                                  norm='slaney'                                            
                                                 )
        return mel_spec
        

    def __call__(self, sample):
        waveform, transcription,sentence = sample['waveform'], sample['transcription'],sample['sentence']
        #zero pad waveform
#         zero_pad = torch.zeros(1, sample_rate*max_time- waveform.size()[1])
#         padding = torch.cat([waveform,zero_pad],1)
        # get spectrogram
        self.hmm = waveform.squeeze()
        wave_spec = self.mel_spectrogram(self.hmm)
        wave_spec = wave_spec.swapaxes(0,1)
        wave_spec = torch.tensor(wave_spec,device=device)
        #change transcription list to tensor
        transcription = torch.tensor(transcription, dtype=torch.long, device=device)

        return {'waveform': wave_spec, 'transcription': transcription, 'sentence':sentence}
    
transformed_dataset = VoiceDataset(transform = MelSpec())

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['waveform'].size(), sample['sentence'])

    if i == 3:
        break

0 torch.Size([658, 80]) six
1 torch.Size([418, 80]) alls well that ends well
2 torch.Size([598, 80]) do you mean it
3 torch.Size([646, 80]) the new patch is less invasive than the old one but still causes regressions


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]    
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
    return batch


def pad_collate(batch):
    max_input_len = float('-inf')
    max_target_len = float('-inf')

    for elem in batch:
        feature = elem['waveform']
        feature = feature.squeeze()
        trn = elem['transcription']
        max_input_len = max_input_len if max_input_len > feature.shape[0] else feature.shape[0]
        max_target_len = max_target_len if max_target_len > len(trn) else len(trn)

    for i, elem in enumerate(batch):
        f = elem['waveform']
        trn = elem['transcription']
        sentence = elem['sentence']
        f = f.squeeze()
        input_length = f.shape[0]
        input_dim = f.shape[1]
        # print('f.shape: ' + str(f.shape))
        feature = np.zeros((max_input_len, input_dim), dtype=np.float32)
        feature[:f.shape[0], :f.shape[1]] = f.cpu()
        trn = np.pad(trn.cpu(), (0, max_target_len - len(trn)), 'constant', constant_values=29)
        batch[i] = (feature, trn, input_length,sentence)
        # print('feature.shape: ' + str(feature.shape))
        # print('trn.shape: ' + str(trn.shape))
    batch.sort(key=lambda x: x[2], reverse=True)

    return default_collate(batch)



def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets,sentence = [], [],[]

    # Gather in lists, and encode labels as indices
    for a in batch:
        tensors += [a['waveform']]
        targets += [a['transcription']]
        sentence += [a['sentence']]
                   
    # Group the list of tensors into a batched tensor
    tensors = tensors
#     targets = torch.stack(targets)
    targets = pad_sequence(targets)

    return tensors, targets,sentence


train_loader = DataLoader(transformed_dataset, batch_size=10,collate_fn=pad_collate,
                        shuffle=True, num_workers=0)

iterator = iter(train_loader)
x_batch,y,input_lengths,sentence = iterator.next()
print(x_batch,y,input_lengths,sentence)

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.4847e-16, 5.2102e-16, 3.3847e-16,  ..., 1.8097e-17,
          1.7571e-17, 1.7281e-17],
         [4.5527e-08, 1.5976e-07, 9.2791e-08,  ..., 1.3166e-09,
          1.0629e-09, 2.0471e-10],
         ...,
         [2.0183e-07, 7.0827e-07, 4.4801e-07,  ..., 3.4974e-09,
          8.2973e-10, 3.0359e-10],
         [5.6650e-06, 1.9879e-05, 1.6335e-05,  ..., 5.4585e-10,
          2.1047e-10, 4.9710e-10],
         [9.3017e-07, 3.2641e-06, 1.8571e-06,  ..., 4.9481e-10,
          2.6179e-10, 1.3402e-10]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [54]:
x_batch.shape

torch.Size([10, 764, 80])

In [55]:
input_lengths

tensor([764, 634, 625, 526, 521, 502, 500, 392, 358, 207])

In [7]:
d1 = transformed_dataset[10]['waveform']

In [36]:
n1 = nn.GRU(80,10)
n2 = nn.GRU(10, 10)
n3 = nn.GRU(10, 10)

In [37]:
o1,o2 = n1(x_batch)
b1,b2 = n2(o1)

In [60]:
hmm = nn.GRU(80,10,bidirectional=True)
hmm2 = nn.GRU(20,5,bidirectional=True)
input_x = x_batch.size(1)
enc_len = x_batch.size(2)
total_length = x_batch.size(1)
packed_input = pack_padded_sequence(x_batch, input_lengths, batch_first=True)
hah, oo= hmm(packed_input,oo)
wq,tt = hmm2(hah)
output, _ = pad_packed_sequence(wq, batch_first=True, total_length=total_length)


In [70]:
def pad_list(xs, pad_value):
        # From: espnet/src/nets/e2e_asr_th.py: pad_list()
        n_batch = len(xs)
        max_len = max(x.size(0) for x in xs)
        pad = xs[0].new(n_batch, max_len, *xs[0].size()[1:]).fill_(pad_value)
        for i in range(n_batch):
            pad[i, :xs[i].size(0)] = xs[i]
        return pad

In [71]:
ys = [yi[yi != 29] for yi in y]  # parse padded ys
# prepare input and output word sequences with sos/eos IDs
eos = ys[0].new([27])
sos = ys[0].new([28])
ys_in = [y for y in ys]
ys_out = [y for y in ys]
# padding for ys with -1
# pys: utt x olen
ys_in_pad = pad_list(ys_in, 27)
ys_out_pad = pad_list(ys_out, 29)
assert ys_in_pad.size() == ys_out_pad.size()
batch_size = ys_in_pad.size(0)
output_length = ys_in_pad.size(1)

In [81]:
y

tensor([[ 9, 20,  0,  3,  1, 14,  0,  8, 15, 23,  5, 22,  5, 18,  0,  2,  5,  0,
         19, 12,  1,  9, 14,  0, 23,  9, 20,  8,  0,  1,  0, 19,  9, 12, 22,  5,
         18,  0,  2, 21, 12, 12,  5, 20],
        [ 9, 12, 12,  0, 15, 23, 14,  0, 20,  8,  1, 20,  0,  8,  5, 19,  0, 23,
          9, 20,  8,  0,  1,  0,  8,  1, 18, 19,  8,  0, 19,  5, 20, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29],
        [ 4, 15,  0, 18,  9,  7,  8, 20,  0,  1, 14,  4,  0,  6,  5,  1, 18,  0,
         14, 15,  0, 13,  1, 14, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29],
        [ 1,  0, 13,  1, 14,  0,  9, 19,  0, 11, 14, 15, 23, 14,  0,  2, 25,  0,
         20,  8,  5,  0,  3, 15, 13, 16,  1, 14, 25,  0,  8,  5,  0, 11,  5,  5,
         16, 19, 29, 29, 29, 29, 29, 29],
        [20,  8,  1, 20,  0, 19, 15,  2,  5, 18,  5,  4,  0,  8,  9, 13,  0,  1,
          0, 12,  9, 20, 20, 12,  5, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 2

In [82]:
ys_in_pad

tensor([[ 9, 20,  0,  3,  1, 14,  0,  8, 15, 23,  5, 22,  5, 18,  0,  2,  5,  0,
         19, 12,  1,  9, 14,  0, 23,  9, 20,  8,  0,  1,  0, 19,  9, 12, 22,  5,
         18,  0,  2, 21, 12, 12,  5, 20],
        [ 9, 12, 12,  0, 15, 23, 14,  0, 20,  8,  1, 20,  0,  8,  5, 19,  0, 23,
          9, 20,  8,  0,  1,  0,  8,  1, 18, 19,  8,  0, 19,  5, 20, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27],
        [ 4, 15,  0, 18,  9,  7,  8, 20,  0,  1, 14,  4,  0,  6,  5,  1, 18,  0,
         14, 15,  0, 13,  1, 14, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27],
        [ 1,  0, 13,  1, 14,  0,  9, 19,  0, 11, 14, 15, 23, 14,  0,  2, 25,  0,
         20,  8,  5,  0,  3, 15, 13, 16,  1, 14, 25,  0,  8,  5,  0, 11,  5,  5,
         16, 19, 27, 27, 27, 27, 27, 27],
        [20,  8,  1, 20,  0, 19, 15,  2,  5, 18,  5,  4,  0,  8,  9, 13,  0,  1,
          0, 12,  9, 20, 20, 12,  5, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 2

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0, bidirectional=True):
        super(Encoder, self).__init__()
        
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout,
                          bidirectional=bidirectional)
        self.rnn2 = nn.GRU(hidden_size*2, int(hidden_size/4), batch_first=True,
                          bidirectional=bidirectional)
        

        

    def forward(self, input_x, enc_len):
        total_length = input_x.size(1)  # get the max sequence length
        # print('total_length: ' + str(total_length))
        # print('input_x.size(): ' + str(input_x.size()))
        packed_input = pack_padded_sequence(input_x, enc_len, batch_first=True,)
        # print('enc_len: ' + str(enc_len))
        packed_output, hidden = self.rnn(packed_input)
#         packed_output, hidden = self.rnn2(packed_output)
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=total_length)
        return output, hidden
    
hmm = nn.GRU(80,20,bidirectional=True)
hmm2 = nn.GRU(40,10,bidirectional=True)


input_x = x_batch.size(1)
enc_len = x_batch.size(2)
total_length =x_batch.size(1)
input_lengths.long().to(device)
packed_input = pack_padded_sequence(x_batch,input_lengths, batch_first=True)
hah, _= hmm(packed_input)
hah,_ = hmm2(hah)
output, _ = pad_packed_sequence(hah, batch_first=True, total_length=total_length)


In [29]:
output.shape

torch.Size([10, 764, 20])

In [65]:
x_batch.device

device(type='cpu')

In [16]:
encoder = Encoder(80, 30, 1, dropout=0.0, bidirectional=False)

gg, oo = encoder(x_batch,input_lengths)

In [17]:
gg.shape

torch.Size([10, 399, 30])

In [18]:
oo.shape

torch.Size([1, 10, 30])

In [191]:
encoder = Encoder(80, 30, 1, dropout=0.0, bidirectional=True)
decoder = Decoder(vocab_size=29, embedding_dim=15, hidden_size=30, num_layers=1)

In [115]:
y.shape

torch.Size([10, 63])

In [192]:
ww, wo ,wi = decoder(y,gg,oo)

In [193]:
ww.shape, wo.shape,wi.shape

(torch.Size([640, 29]), torch.Size([640]), torch.Size([10, 64]))

In [223]:
hmm = Fi.cross_entropy(ww, wo,
                                  ignore_index=29,
                                  reduction='none')

In [211]:
hmm.mean()

tensor(1.9204, grad_fn=<MeanBackward0>)

In [221]:
hmm

tensor(3.3766, grad_fn=<NllLossBackward>)

In [226]:
hmm[hmm >0].shape

torch.Size([364])

In [222]:
1229/3.3766

363.9755967541314

In [157]:
# rnny = nn.ModuleList()
rnny = nn.LSTMCell(15, 30,)

In [188]:
em = decoder.embedding(y)

IndexError: index out of range in self

ln = em.size(1)

In [160]:
wah = rnny(em[:,0,:],(oo[0],oo[1]))

In [163]:
wah[2].shape

IndexError: tuple index out of range

In [148]:
em[:,i,:].shape

torch.Size([10, 15])

In [146]:
for i in range(em.size(1)):
    h,c = rnny(em[:,i,:])

In [7]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=1):
        super(Decoder, self).__init__()
        # Hyper parameters
        # embedding + output
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        # rnn
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.encoder_hidden_size = hidden_size  # must be equal now
        # Components
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.ModuleList()
        self.gru += [nn.LSTMCell(self.embedding_dim, self.hidden_size)]
        for l in range(1, self.num_layers):
            self.gru += [nn.LSTMCell(self.hidden_size, self.hidden_size)]
        self.mlp = nn.Sequential(
            nn.Linear(self.encoder_hidden_size,
                      self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.vocab_size))

    def zero_state(self, encoder_padded_outputs, H=None):
        N = encoder_padded_outputs.size(0)
        H = self.hidden_size if H == None else H
        return encoder_padded_outputs.new_zeros(N, H)
    
    def pad_list(self,xs, pad_value):
        # From: espnet/src/nets/e2e_asr_th.py: pad_list()
        n_batch = len(xs)
        max_len = max(x.size(0) for x in xs)
        pad = xs[0].new(n_batch, max_len, *xs[0].size()[1:]).fill_(pad_value)
        for i in range(n_batch):
            pad[i, :xs[i].size(0)] = xs[i]
        return pad
    
    def get_pads(self,padded_input):
        PAD_token = dictOfchar['PAD']
        EOS_token = dictOfchar['EOS']
        SOS_token = dictOfchar['SOS']
        ys = [y[y != PAD_token] for y in padded_input]  # parse padded ys
        # prepare input and output word sequences with sos/eos IDs
        eos = ys[0].new([EOS_token])
        sos = ys[0].new([SOS_token])
        ys_in = [torch.cat([sos, y], dim=0) for y in ys]
        ys_out = [torch.cat([y, eos], dim=0) for y in ys]
#         padding for ys with -1
        # pys: utt x olen
        ys_in_pad = self.pad_list(ys, EOS_token)
        ys_out_pad = self.pad_list(ys, PAD_token)
        assert ys_in_pad.size() == ys_out_pad.size()
        
        return ys_in_pad, ys_out_pad

    def forward(self, padded_input, encoder_padded_outputs,encoder_hidden):
        """
        Args:
            padded_input: N x To
            # encoder_hidden: (num_layers * num_directions) x N x H
            encoder_padded_outputs: N x Ti x H
        Returns:
        """
        # *********Get Input and Output
        # from espnet/Decoder.forward()
        
#         ys_in_pad, ys_out_pad = self.get_pads(padded_input)
        
        ys_in_pad = padded_input
        ys_out_pad = padded_input

        batch_size = ys_in_pad.size(0)
        output_length = ys_in_pad.size(1)
             
        # *********Init decoder rnn
        h_list = [encoder_hidden[0]]
        c_list = [encoder_hidden[1]]
        
        for l in range(1, self.num_layers):
            h_list.append(self.zero_state(encoder_padded_outputs))
            c_list.append(self.zero_state(encoder_padded_outputs))

        y_all = []
        

        # **********LAS: 1. decoder rnn 2. attention 3. concate and MLP
        embedded = self.embedding(ys_in_pad)
        for t in range(output_length):
            # step 1. decoder RNN: s_i = RNN(s_i−1,y_i−1,c_i−1)

            rnn_input = embedded[:, t, :]

            h_list[0], c_list[0] = self.gru[0](
                rnn_input, (h_list[0], c_list[0]))
            for l in range(1, self.num_layers):
                h_list[l], c_list[l] = self.gru[l](
                    h_list[l - 1], (h_list[l], c_list[l]))
            rnn_output = h_list[-1]
            mlp_input = rnn_output
            predicted_y_t = self.mlp(mlp_input)
            y_all.append(predicted_y_t)

        y_all = torch.stack(y_all, dim=1)  # N x To x C

        # **********Cross Entropy Loss
        # F.cross_entropy = NLL(log_softmax(input), target))
        y_all = y_all.view(batch_size * output_length, self.vocab_size)
#         ce_loss = F.cross_entropy(y_all, ys_out_pad.view(-1),
#                                   ignore_index=PAD_token,
#                                   reduction='mean')

        return y_all, ys_out_pad.view(-1)


In [7]:
class DotProductAttention(nn.Module):
    r"""Dot product attention.
    Given a set of vector values, and a vector query, attention is a technique
    to compute a weighted sum of the values, dependent on the query.
    NOTE: Here we use the terminology in Stanford cs224n-2018-lecture11.
    """

    def __init__(self):
        super(DotProductAttention, self).__init__()

    def forward(self, queries, values):
        """
        Args:
            queries: N x To x H
            values : N x Ti x H
        Returns:
            output: N x To x H
            attention_distribution: N x To x Ti
        """
        batch_size = queries.size(0)
        input_lengths = values.size(1)
        # (N, To, H) * (N, H, Ti) -> (N, To, Ti)
        attention_scores = torch.bmm(queries, values.transpose(1, 2))
        attention_distribution = Fi.softmax(
            attention_scores.view(-1, input_lengths), dim=1).view(batch_size, -1, input_lengths)
        # (N, To, Ti) * (N, Ti, H) -> (N, To, H)
        attention_output = torch.bmm(attention_distribution, values)

        return attention_output, attention_distribution


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=1):
        super(Decoder, self).__init__()
        # Hyper parameters
        # embedding + output
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        # rnn
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.encoder_hidden_size = hidden_size  # must be equal now
        # Components
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.rnn = nn.ModuleList()
        self.rnn += [nn.LSTMCell(self.embedding_dim +
                                 self.encoder_hidden_size, self.hidden_size)]
        for l in range(1, self.num_layers):
            self.rnn += [nn.LSTMCell(self.hidden_size, self.hidden_size)]
        self.attention = DotProductAttention()
        self.mlp = nn.Sequential(
            nn.Linear(self.encoder_hidden_size + self.hidden_size,
                      self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.vocab_size))

    def zero_state(self, encoder_padded_outputs, H=None):
        N = encoder_padded_outputs.size(0)
        H = self.hidden_size if H == None else H
        return encoder_padded_outputs.new_zeros(N, H)
    
    def pad_list(self,xs, pad_value):
        # From: espnet/src/nets/e2e_asr_th.py: pad_list()
        n_batch = len(xs)
        max_len = max(x.size(0) for x in xs)
        pad = xs[0].new(n_batch, max_len, *xs[0].size()[1:]).fill_(pad_value)
        for i in range(n_batch):
            pad[i, :xs[i].size(0)] = xs[i]
        return pad

    def forward(self, padded_input, encoder_padded_outputs):
        """
        Args:
            padded_input: N x To
            # encoder_hidden: (num_layers * num_directions) x N x H
            encoder_padded_outputs: N x Ti x H
        Returns:
        """
        # *********Get Input and Output
        # from espnet/Decoder.forward()
        ys = [y[y != PAD_token] for y in padded_input]  # parse padded ys
        # prepare input and output word sequences with sos/eos IDs

        # padding for ys with -1
        # pys: utt x olen
        sos = ys[0].new([SOS_token])
        eos = ys[0].new([EOS_token])
        ys_in = [torch.cat([sos, y], dim=0) for y in ys]
        ys_out = [torch.cat([y, eos], dim=0) for y in ys]
     
        ys_in_pad = self.pad_list(ys_in, EOS_token)
        ys_out_pad = self.pad_list(ys_out, PAD_token)

        assert ys_in_pad.size() == ys_out_pad.size()
        batch_size = ys_in_pad.size(0)
        output_length = ys_in_pad.size(1)

        # *********Init decoder rnn
        h_list = [self.zero_state(encoder_padded_outputs)]
        c_list = [self.zero_state(encoder_padded_outputs)]
        for l in range(1, self.num_layers):
            h_list.append(self.zero_state(encoder_padded_outputs))
            c_list.append(self.zero_state(encoder_padded_outputs))
        att_c = self.zero_state(encoder_padded_outputs,
                                H=encoder_padded_outputs.size(2))
        y_all = []

        # **********LAS: 1. decoder rnn 2. attention 3. concate and MLP
        embedded = self.embedding(ys_in_pad)
        for t in range(output_length):
            # step 1. decoder RNN: s_i = RNN(s_i−1,y_i−1,c_i−1)
            rnn_input = torch.cat((embedded[:, t, :], att_c), dim=1)
#             print(rnn_input.size(),embedded.size(),att_c.size())
            h_list[0], c_list[0] = self.rnn[0](
                rnn_input, (h_list[0], c_list[0]))
            for l in range(1, self.num_layers):
                h_list[l], c_list[l] = self.rnn[l](
                    h_list[l - 1], (h_list[l], c_list[l]))
            rnn_output = h_list[-1]  # below unsqueeze: (N x H) -> (N x 1 x H)
            # step 2. attention: c_i = AttentionContext(s_i,h)
            att_c, att_w = self.attention(rnn_output.unsqueeze(dim=1),
                                          encoder_padded_outputs)
            att_c = att_c.squeeze(dim=1)
            # step 3. concate s_i and c_i, and input to MLP
            mlp_input = torch.cat((rnn_output, att_c), dim=1)
            predicted_y_t = self.mlp(mlp_input)
            y_all.append(predicted_y_t)

        y_all = torch.stack(y_all, dim=1)  # N x To x C

        # **********Cross Entropy Loss
        # F.cross_entropy = NLL(log_softmax(input), target))
        y_all = y_all.view(batch_size * output_length, self.vocab_size)
#         ce_loss = F.cross_entropy(y_all, ys_out_pad.view(-1),
#                                   ignore_index=PAD_token,
#                                   reduction='mean')

        return y_all, ys_out_pad.view(-1)

    def recognize_beam(self, encoder_outputs, beam =1,nbest=3):
        """Beam search, decode one utterence now.
        Args:
            encoder_outputs: T x H
            char_list: list of character
            args: args.beam
        Returns:
            nbest_hyps:
        """
        maxlen = 100

        # *********Init decoder rnn
        h_list = [self.zero_state(encoder_outputs.unsqueeze(0))]
        c_list = [self.zero_state(encoder_outputs.unsqueeze(0))]
        for l in range(1, self.num_layers):
            h_list.append(self.zero_state(encoder_outputs.unsqueeze(0)))
            c_list.append(self.zero_state(encoder_outputs.unsqueeze(0)))
        att_c = self.zero_state(encoder_outputs.unsqueeze(0),
                                H=encoder_outputs.unsqueeze(0).size(2))
        # prepare sos
        y = SOS_token
        vy = encoder_outputs.new_zeros(1).long()

        hyp = {'score': 0.0, 'yseq': [y], 'c_prev': c_list, 'h_prev': h_list,
               'a_prev': att_c}
        hyps = [hyp]
        ended_hyps = []

        for i in range(maxlen):
            hyps_best_kept = []
            for hyp in hyps:
                # vy.unsqueeze(1)
                vy[0] = hyp['yseq'][i]
                embedded = self.embedding(vy)
                # embedded.unsqueeze(0)

                # step 1. decoder RNN: s_i = RNN(s_i−1,y_i−1,c_i−1)
                rnn_input = torch.cat((embedded, hyp['a_prev']), dim=1)
                h_list[0], c_list[0] = self.rnn[0](
                    rnn_input, (hyp['h_prev'][0], hyp['c_prev'][0]))
                for l in range(1, self.num_layers):
                    h_list[l], c_list[l] = self.rnn[l](
                        h_list[l - 1], (hyp['h_prev'][l], hyp['c_prev'][l]))
                rnn_output = h_list[-1]

                # step 2. attention: c_i = AttentionContext(s_i,h)
                # below unsqueeze: (N x H) -> (N x 1 x H)
                att_c, att_w = self.attention(rnn_output.unsqueeze(dim=1),
                                              encoder_outputs.unsqueeze(0))
                att_c = att_c.squeeze(dim=1)

                # step 3. concate s_i and c_i, and input to MLP
                mlp_input = torch.cat((rnn_output, att_c), dim=1)
                predicted_y_t = self.mlp(mlp_input)
                local_scores = Fi.log_softmax(predicted_y_t, dim=1)
                # topk scores
                local_best_scores, local_best_ids = torch.topk(
                    local_scores, beam, dim=1)

                for j in range(beam):
                    new_hyp = {}
                    new_hyp['h_prev'] = h_list[:]
                    new_hyp['c_prev'] = c_list[:]
                    new_hyp['a_prev'] = att_c[:]
                    new_hyp['score'] = hyp['score'] + local_best_scores[0, j]
                    new_hyp['yseq'] = [0] * (1 + len(hyp['yseq']))
                    new_hyp['yseq'][:len(hyp['yseq'])] = hyp['yseq']
                    new_hyp['yseq'][len(hyp['yseq'])] = int(
                        local_best_ids[0, j])
                    # will be (2 x beam) hyps at most
                    hyps_best_kept.append(new_hyp)

                hyps_best_kept = sorted(hyps_best_kept,
                                        key=lambda x: x['score'],
                                        reverse=True)[:beam]
            # end for hyp in hyps
            hyps = hyps_best_kept

            # add eos in the final loop to avoid that there are no ended hyps
            if i == maxlen - 1:
                for hyp in hyps:
                    hyp['yseq'].append(EOS_token)

            # add ended hypothes to a final list, and removed them from current hypothes
            # (this will be a probmlem, number of hyps < beam)
            remained_hyps = []
            for hyp in hyps:
                if hyp['yseq'][-1] == EOS_token:
                    # hyp['score'] += (i + 1) * penalty
                    ended_hyps.append(hyp)
                else:
                    remained_hyps.append(hyp)

            hyps = remained_hyps
            if len(hyps) > 0:
                pass
                # print('remeined hypothes: ' + str(len(hyps)))
            else:
                # print('no hypothesis. Finish decoding.')
                break

            # for hyp in hyps:
            #     print('hypo: ' + ''.join([char_list[int(x)]
            #                               for x in hyp['yseq'][1:]]))
        # end for i in range(maxlen)
        nbest_hyps = sorted(ended_hyps, key=lambda x: x['score'], reverse=True)[
                     :min(len(ended_hyps), nbest)]
        return nbest_hyps


In [11]:
PAD_token = dictOfchar['PAD']
EOS_token = dictOfchar['EOS']
SOS_token = dictOfchar['SOS']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yuck =[]
hmm = []
jk=[]

def train(features, encoder, decoder, optimizer, criterion):
    encoder.train()
    decoder.train()

    optimizer.zero_grad()
    
    
    input_tensor = features[0].float().to(device)
    target_tensor = features[1].long().to(device)
    input_length = features[2].long().to(device)
        
    
    batch_size = input_tensor.size(0)
    
    encoder_output, encoder_hidden = encoder(input_tensor,input_length)
    pred,actual = decoder(target_tensor,encoder_output)
    loss = criterion(pred,actual,ignore_index=PAD_token,reduction='mean')
#     loss = loss[loss>0].sum()/batch_size
# #     print(loss)
# #     yuck.append(criterion(pred,actual,ignore_index=PAD_token,reduction='none'))
  
    loss.backward()
    optimizer.step()
        

    return loss.item()


def validate(features, encoder, decoder, optimizer, criterion):
    encoder.eval()
    decoder.eval()

    
    
    input_tensor = features[0].float().to(device)
    target_tensor = features[1].long().to(device)
    input_length = features[2].long().to(device)
        
    
    batch_size = input_tensor.size(0)
    
    encoder_output, encoder_hidden = encoder(input_tensor,input_length)
    pred,actual = decoder(target_tensor,encoder_output)
    loss = criterion(pred,actual,ignore_index=PAD_token,reduction='mean')
#     loss = loss[loss>0].sum()/batch_size
# #     print(loss)
# #     yuck.append(criterion(pred,actual,ignore_index=PAD_token,reduction='none'))

    return loss

In [8]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [23]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

plot_losses = []
plot_losses_val = []

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
    
def save_checkpoint(epoch, encoder, decoder, optimizer, loss):
    torch.save({
            'epoch': epoch,
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '../models/output/model.pth')


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.001,reload=False):
    start = time.time()
    plot_losses = []
    plot_losses_val = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print_loss_total_val = 0
    plot_loss_total_val =0
    
    
    lns = len(transformed_dataset)
    optimizer = torch.optim.Adam([{'params': encoder.parameters()}, {'params': decoder.parameters()}],
                                         lr=learning_rate)
    
    if reload:
        print("loading previously trained model")
        checkpoint = torch.load('../models/output/model.pth',map_location='cpu')
        encoder.load_state_dict(checkpoint['encoder_state_dict'])
        encoder = encoder.to(device)
        decoder.load_state_dict(checkpoint['decoder_state_dict'])
        decoder = decoder.to(device)
        encoder.train()
        decoder.train()
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        del checkpoint
        torch.cuda.empty_cache()
        
        

                      
#     criterion = nn.NLLLoss()
    criterion = Fi.cross_entropy
    len_all = len(transformed_dataset)
    len_train = int(0.8*len_all)
    train_ds, val_ds = torch.utils.data.random_split(transformed_dataset, (len_train,len_all-len_train ))

    


    for i in range(1, n_iters):
        rand_sampler = torch.utils.data.RandomSampler(val_ds)
        train_sampler = DataLoader(train_ds, batch_size=32, sampler=rand_sampler,collate_fn=pad_collate)
        
#         rand_val_sampler = torch.utils.data.RandomSampler(val_ds)
#         val_sampler = DataLoader(transformed_dataset, batch_size=10, sampler=rand_val_sampler,collate_fn=pad_collate)
        
        iterator = iter(train_sampler)        
        features = iterator.next()            
        loss = train(features, encoder, decoder, optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        
#         iter_val = iter(val_sampler)        
#         features_val = iter_val.next()            
#         loss_val = validate(features_val, encoder, decoder, optimizer, criterion)
#         print_loss_total_val += loss_val
#         plot_loss_total_val += loss_val

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            
            print_loss_avg_val = print_loss_total_val / print_every
            print_loss_total_val = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, i/ n_iters),
                                         i, i / n_iters * 100, print_loss_avg))
            plot_losses.append(print_loss_avg)
#             plot_losses_val.append(print_loss_avg_val)
            save_checkpoint(i, encoder, decoder, optimizer, loss)
            
           
            


# Train model

In [91]:
rand_sampler = torch.utils.data.RandomSampler(transformed_dataset, num_samples=10, replacement=True)
train_sampler = DataLoader(transformed_dataset, batch_size=10, sampler=rand_sampler,collate_fn=pad_collate)

In [ ]:
DataLoader(transformed_dataset, batch_size=10,collate_fn=pad_collate,
                        shuffle=True, num_workers=0)

In [23]:
torch.cuda.empty_cache()

In [44]:
enc_path = "enc_model_las_all"
torch.save(encoder, enc_path)
dec_path = "dec_model_las_all"
torch.save(decoder, dec_path)

/home/ubuntu/anaconda3/envs/oldtorch/lib/python3.8/site-packages/torch/serialization.py:401: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
/home/ubuntu/anaconda3/envs/oldtorch/lib/python3.8/site-packages/torch/serialization.py:401: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
/home/ubuntu/anaconda3/envs/oldtorch/lib/python3.8/site-packages/torch/serialization.py:401: UserWarning: Couldn't retrieve source code for container of type DotProductAttention. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "


In [50]:
encoder1 = Encoder(80,40, 2, dropout=0.1, bidirectional=True).to(device)


In [51]:
encoder1

Encoder(
  (rnn): GRU(80, 40, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (rnn2): GRU(80, 10, batch_first=True, dropout=0.1, bidirectional=True)
)

In [35]:
enc_path = "../models/output/enc_model_las"
torch.save(encoder.state_dict(), enc_path)
dec_path = "../models/output/dec_model_las"
torch.save(decoder.state_dict(), dec_path)

In [29]:
trainIters(encoder,decoder, 1000, print_every=100, plot_every=2000,learning_rate=0.00005)

0m 53s (- 8m 5s) (100 10%) 0.4845
1m 49s (- 7m 18s) (200 20%) 0.4901
2m 43s (- 6m 22s) (300 30%) 0.4709
3m 38s (- 5m 27s) (400 40%) 0.4790
4m 32s (- 4m 32s) (500 50%) 0.4745
5m 26s (- 3m 37s) (600 60%) 0.4681
6m 19s (- 2m 42s) (700 70%) 0.4742
7m 13s (- 1m 48s) (800 80%) 0.4686
8m 6s (- 0m 54s) (900 90%) 0.4647


In [33]:
import pickle

In [34]:
with open('../models/output/encoder_vars_las.pkl', 'wb') as convert_file:
    pickle.dump(encoder,convert_file)

with open('../models/output/decoder_vars_las.pkl', 'wb') as convert_file:
    pickle.dump(decoder,convert_file)

/home/ubuntu/anaconda3/envs/oldtorch/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
encoder = Encoder(80,128, 2, dropout=0.1, bidirectional=True).to(device)
decoder = Decoder(vocab_size=29, embedding_dim=50, hidden_size=256, num_layers=4).to(device)

trainIters(encoder,decoder, 20000, print_every=1000, plot_every=100,learning_rate=0.001,reload=False)

10m 5s (- 191m 47s) (1000 5%) 2.0285
20m 5s (- 180m 51s) (2000 10%) 1.5040
30m 10s (- 170m 56s) (3000 15%) 1.3871
40m 4s (- 160m 16s) (4000 20%) 1.3017
49m 59s (- 149m 58s) (5000 25%) 1.2026
59m 55s (- 139m 48s) (6000 30%) 1.0730
69m 55s (- 129m 52s) (7000 35%) 0.9549


In [56]:
trainIters(encoder,decoder, 2000, print_every=100, plot_every=2000,learning_rate=0.005)

1m 39s (- 31m 26s) (100 5%) 95.2875
3m 18s (- 29m 46s) (200 10%) 94.9309
4m 57s (- 28m 5s) (300 15%) 94.9917
6m 35s (- 26m 23s) (400 20%) 94.6034
8m 16s (- 24m 48s) (500 25%) 94.2793


KeyboardInterrupt: 

In [37]:
def evaluate(encoder, decoder, features, max_length=100, beam=1, nbest=1):
    with torch.no_grad():
        input_tensor = features[0]
        target_tensor = features[1]
        input_length = features[2]
        
        input_tensor = features[0].float().to(device)
        target_tensor = features[1].long().to(device)
        input_length = features[2].long().to(device)
        
#         checkpoint = torch.load('../models/output/model.pth',map_location=device)
#         encoder.load_state_dict(checkpoint['encoder_state_dict'])
        encoder.eval()
#         decoder.load_state_dict(checkpoint['decoder_state_dict'])
        decoder.eval()
        
        decoded_words = []


        encoder_outputs, _ = encoder(input_tensor, input_length)
        nbest_hyps = decoder.recognize_beam(encoder_outputs[0], beam, nbest)
        if beam > 1:
            word_index = nbest_hyps[0]['yseq']
        else:
            word_index = nbest_hyps[0]['yseq']
        decoded_word = [dictOfindex[a] for a in word_index]

    return decoded_word

def evaluateRandomly(transformed_dataset,encoder, decoder, n=1):
    samp_1 = DataLoader(transformed_dataset, batch_size=1, collate_fn=pad_collate,
                        shuffle=True, num_workers=0)
    iterator = iter(samp_1)
    samp = iterator.next()

    for i in range(n):
        iterator = iter(samp_1)
        samp = iterator.next()
        actual = samp[3]
        output_words = evaluate(encoder, decoder, samp, max_length=100)
        output_sentence = ' '.join(output_words)
        print(actual, '<', output_sentence)
        print('')
        
evaluateRandomly(transformed_dataset,encoder, decoder, n=10)

('sentences are built of nouns verbs and sometimes adjectives',) < SOS s e v e n   s i z z a   b e a r d   o f   n o s e   v o t e s   a n d   s o m e t i m e s   a d j e c t o r s EOS

('add tomorrow is a long time to my playlist sueos',) < SOS i   h a v e   t o   m o r e   a n y t h i n g   t o   r e p l y   t h e   l i s t   s l a m EOS

('he made it a habit to sell the installed organ after a short while',) < SOS h e   m a r r i e d   t h e   h e a v e d   t o   s e r v e   t h e   i n s t a l l e d   o v e r   a n   a f t e r n o o n   s h r u b   w i n e EOS

('because my eyes are not yet accustomed to the desert the boy said',) < SOS b u t   t h e   l i g h t   i s   a n o t h e r   s u c c e s s f u l   t o   t h e   d e s e r t   t h e   b o y   s a i d EOS

('got anything new on the hanging',) < SOS g o t   e n e r g y   n e a r   l o n g   a n d   i   n e e d EOS

('two boys wearing matching blue polo shirts strike a goofy pose',) < SOS t o   b o y s   r a r e   m a t c h   

In [55]:
def evaluate(encoder, decoder, features, max_length=100):
    with torch.no_grad():
        input_tensor = samp[0]
        target_tensor = samp[1]
        input_length = samp[2]
        decoded_words = []
        
        input_tensor = features[0].float().to(device)
        target_tensor = features[1].long().to(device)
        input_length = features[2].long().to(device)
        

        encoder_output, encoder_hidden = encoder(input_tensor,input_length)
        dec_input = torch.tensor([[SOS_token]], device=device)  # SOS
        h_list = [encoder_hidden[0]] 
        c_list = [encoder_hidden[1]]
        
#         for l in range(1, decoder.num_layers):
#             h_list.append(decoder.zero_state(encoder_output.unsqueeze(0)))
#             c_list.append(decoder.zero_state(encoder_output.unsqueeze(0)))

        for c in range(max_length):
            decoder_input = decoder.embedding(dec_input)
#             print(decoder_input.shape)
#             print(decoder_input.shape)
            h_list[0], c_list[0] = decoder.gru[0](decoder_input[:, 0, :], (h_list[0], c_list[0]))
            
#             for l in range(1, decoder.num_layers):
#                 h_list[l], c_list[l] = decoder.gru[l](
#                     h_list[l - 1], (h_list[l], c_list[l]))
            
            mlp_input = h_list[-1]
            predicted_y_t = decoder.mlp(mlp_input)
            local_scores = Fi.log_softmax(predicted_y_t, dim=1)
            # topk scores
            topv, topi  = torch.topk(local_scores,1,dim=1)
#             print(torch.topk(local_scores,1,dim=1))
            output_probs = torch.exp(local_scores)
            if topi.item() == EOS_token:
                decoded_words.append('EOS')
                break
            else:
                decoded_words.append(dictOfindex[topi.item()])
            dec_input = topi
    return decoded_words

In [369]:
PATH = "/Users/dami.osoba/work/bawk/models/dec_model_new"
torch.save(decoder, PATH)

In [370]:
PATH = "/Users/dami.osoba/work/bawk/models/enc_model_new"
torch.save(encoder, PATH)

In [50]:
enmodel = torch.load("/Users/dami.osoba/work/bawk/models/enc_model")
enmodel.eval()

decmodel = torch.load("/Users/dami.osoba/work/bawk/models/dec_model")
decmodel.eval()

AttnDecoderRNN(
  (embedding): Embedding(29, 20)
  (attn): Linear(in_features=40, out_features=401, bias=True)
  (attn_combine): Linear(in_features=40, out_features=20, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(20, 20)
  (out): Linear(in_features=20, out_features=29, bias=True)
)

In [59]:
samp_1 = DataLoader(transformed_dataset, batch_size=1,collate_fn=pad_collate,
                        shuffle=True, num_workers=0)
iterator = iter(samp_1) 
samp = iterator.next()
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        iterator = iter(samp_1) 
        samp = iterator.next()
        actual = samp[3]
        output_words = evaluate(encoder, decoder, samp, max_length=100)
        output_sentence = ' '.join(output_words[1:-1])
        print(actual, '<', output_sentence)
        print('')

In [350]:
samp[3]

('leko resigned when checkmate was threatened',)

In [60]:
evaluateRandomly(encoder, decoder, n=10)

('march the soldiers past the next hill',) < r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r  

('what else are you',) < r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r  

('and id better read your books said the boy',) < r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r  

('he personally participated in the crusade against egypt',) < r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r   r  

('current board member of get

In [79]:
train_text[23]

'that might be all right.'

In [77]:
predict(train_path[23],train_text[23],enmodel,decmodel)

that might be all right < SOSthe dige tone<EOS>



In [76]:
def predict(wav_path,transcription,encoder,decoder):
    # Use the model to predict the label of the waveform
    waveform, _ = torchaudio.load(wav_path)
    sentence = transcription.encode(encoding="ascii",errors="ignore").decode().translate(table_trans)
    chars =[b for a in sentence for b in a]
    coded = [28]+[char_dict.index(a) for a in chars]+[27]
    sample['waveform'] =waveform
    sample['transcription'] = coded
    sample['sentence'] = sentence
    transformer = MelSpec()
    mels =transformer(sample)
    ex =mels['waveform']
    
    output_words, attentions,_ = evaluate(encoder, decoder, ex)
    output_sentence = ''.join(output_words)
    return output_sentence
    

In [67]:
def evaluate(encoder, decoder, tens, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tens
        input_length = input_tensor.size(2)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,1,80*401), encoder_hidden,MAX_LENGTH)


        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_output = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention,decoder_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            yay = torch.distributions.categorical.Categorical(decoder_probs)
            topi = yay.sample()
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dictOfindex[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1],decoder_output

In [34]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        choice = np.random.randint(200)
        print(choice)
        actual = transformed_dataset[choice]['sentence']
        ex = transformed_dataset[choice]['waveform']
        output_words, attentions,_ = evaluate(encoder, decoder, ex)
        output_sentence = ''.join(output_words)
        print(actual, '<', output_sentence)
        print('')

# Really bad model

In [39]:
evaluateRandomly(encoder1, attn_decoder1)


168
you said it to me too < SOSthet hy one ill ster a troulenpf<EOS>

183
now look what youve done < SOScoof honing ipler<EOS>

13
two < SOSyo lrlieb cowiovere zoon<EOS>

62
he disappeared into the tent < SOShegiver a of a the marze her erpeesen up eaniap browndoyrewid for adyorcrawntent witt dearntnged on u<EOS>

166
ill talk to you tonight < SOSyou poet fee hy odous courldengtas i sotellndingty on feny ontouenilhan<EOS>

27
how did it happen < SOSthee loong uperee right spigel ven<EOS>

157
richard has gone camping by himself < SOShe it fakrieve auntpy a seaits salingedirs t neees feccid<EOS>

170
what shall we do now < SOSwhot mir they inrey<EOS>

152
good weekend is edited by amelia lester < SOSjex do ge goes lhth<EOS>

11
say what you have got to say < SOSser<EOS>

